In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import nltk.stem as stemmer
from nltk.stem import *
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Ashik/Thesis/'
df = pd.read_csv(f'{path}data/dataset-2.csv')
df.shape

Mounted at /content/drive


(2111, 3)

In [5]:
from nltk.stem.porter import PorterStemmer
import gensim
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api
from nltk import sent_tokenize
from gensim.utils import simple_preprocess


ps = PorterStemmer()

def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    cleaned_text = ' '.join(stemmed_tokens)
    return cleaned_text

df['description'] = df ['description'].apply(clean_text)
text = df['description']
category = df['category']

wv = api.load('word2vec-google-news-300')

corpus = []
for i in range(len(text)):
    x = text[i]
    temp = re.sub('[^a-zA-Z0-9]',' ',x)  # Removing Unnecessary words
    temp = temp.lower()  # Converting them to lowercase
    temp = temp.split()  # Splitting the words

    # Appending words not found in stopwords
    temp = [ps.stem(word) for word in temp if not word in stopwords.words('english')]
    temp = ' '.join(temp)
    corpus.append(temp)


words = []
for sent in corpus:
  sent_token = sent_tokenize(sent) ## Tokenization
  for sent in sent_token:
    words.append(simple_preprocess(sent)) ## Lowering the words


model = gensim.models.Word2Vec(words,window=5,min_count=2)
w2v_model = model

sent_tokenize(corpus[0])


# Now lets conver the output category into numeric by encoding them
y = pd.get_dummies(df['category'],columns=df["category"]).values



# Split the data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(corpus,y,test_size=0.3,random_state=24)


train_words = []
for sent in x_train:
  sent_token = sent_tokenize(sent) ## Tokenization
  for sent in sent_token:
    train_words.append(simple_preprocess(sent)) ## Lowering the words

model = gensim.models.Word2Vec(train_words,window=5,min_count=2)


# Define the tokenizer
from keras.preprocessing.text import Tokenizer
maxlen = 3000
tokenizer = Tokenizer(num_words=50000, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index


x_train_sequences = tokenizer.texts_to_sequences(x_train)
x_test_sequences = tokenizer.texts_to_sequences(x_test)

print(len(x_train_sequences[0]),len(x_train_sequences[1]))

<ipython-input-5-442ba4cda239>:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-5-442ba4cda239>:12: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


[==================================================] 100.0% 1662.8/1662.8MB downloaded
68 41


In [6]:
# Since the number representation of the text are not the same, we need to pad them in order to make them same
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras import backend as K


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


x_train = pad_sequences(x_train_sequences, maxlen=3000,padding='pre')
x_test = pad_sequences(x_test_sequences,maxlen=3000,padding='pre')


model=Sequential()
model.add(Embedding(50000,100,input_length=3000))  # Input as the number of words (vocabulary size)
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2))) #Softmax for multiclass
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=['accuracy',f1_m,precision_m, recall_m])

In [7]:
history = model.fit(x_train,y_train, epochs=7, batch_size=64,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/7
21/21 [==============================] - 441s 21s/step - loss: 1.4705 - accuracy: 0.3679 - f1_m: 0.1534 - precision_m: 0.4899 - recall_m: 0.0959 - val_loss: 1.2643 - val_accuracy: 0.4865 - val_f1_m: 0.1896 - val_precision_m: 0.8333 - val_recall_m: 0.1104
Epoch 2/7
21/21 [==============================] - 439s 21s/step - loss: 1.1479 - accuracy: 0.5237 - f1_m: 0.4412 - precision_m: 0.7555 - recall_m: 0.3307 - val_loss: 0.9774 - val_accuracy: 0.5946 - val_f1_m: 0.5492 - val_precision_m: 0.7720 - val_recall_m: 0.4292
Epoch 3/7
21/21 [==============================] - 441s 21s/step - loss: 0.8932 - accuracy: 0.6388 - f1_m: 0.5756 - precision_m: 0.8095 - recall_m: 0.4487 - val_loss: 0.8248 - val_accuracy: 0.6824 - val_f1_m: 0.5893 - val_precision_m: 0.8410 - val_recall_m: 0.4562
Epoch 4/7
21/21 [==============================] - 442s 21s/step - loss: 0.6523 - accuracy: 0.7344 - f1_m: 0.7013 - precision_m: 0.8697 - recall_m: 0.5911 - val_loss: 0.7839 - val_accuracy: 0.7162 - val_f1

In [8]:
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=0)

In [9]:
print(loss, accuracy, f1_score, precision, recall)

0.7782526016235352 0.7176656126976013 0.7090528607368469 0.7273728251457214 0.6920672655105591


In [10]:
# #bi-directional
# model=Sequential()
# model.add(Embedding(50000,100,input_length=3000))  # Input as the number of words (vocabulary size)
# model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2))) #Softmax for multiclass
# model.add(TimeDistributed(Dense(5, activation='softmax')))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', auroc])